In [ ]:
# Este trecho de cógido é utilizado para obter resultados de medições do tipo traceroute já realizadas
# Os ids e os alvos das medições ficam armazenados no arquivo measurement_traceroute

import requests
import json
import sqlite3
import multiprocessing as mp
import os
from datetime import datetime,timedelta
from datetime import date
import time
import bisect
import tqdm
url = 'https://atlas.ripe.net:443/api/v2/measurements/'
ips0 = set()

a = list()
with open("measurement_traceroute",'r') as fp:
    for i in fp:
        r = i.split(" ")
        a.append((r[0],r[1]))

for measurement,dns in a:
    target = dict()    
    resposta = requests.get(url+str(measurement)+"/results")
    resposta = resposta.json()
    for probe in resposta:
        target[probe["from"]]= list()
        for result1 in probe["result"]:
            for result in result1["result"]:
                if "from" in result:
                    target[probe["from"]].append((result["from"],result["ttl"],result["rtt"]))
                    ips0.add(result["from"])
                else:
                    target[probe["from"]].append((result["x"],result["x"],result["x"]))
                break
            # 
    # print(target)
    with open("rDNSmaster/database",'w') as log:
        for i in ips0:
            print(i,file=log)
    with open("rDNSmaster/trace.prb",'r') as ds:
        ips = dict()
        for i in ds:
            gh = i.split(' ,')
            ips[gh[0]] = gh[1][:-1]
        # print(ips)
    with open("Traceroute/Level3/"+ dns[:-1]+".traceroute",'w') as df:
        for i in target:
            print( "Probe " + i,file=df)
            print(dns[:-1])
            print( "Probe " + i)
            count = 1
            for j in target[i]:
                if j[0] in ips:
                    print(f"hop = {count}  IP = {j[0]}   rDNS = {ips[j[0]]}  TTL = {j[1]}  rtt = {j[2]}",file=df)
                    print(f"{count}  & {j[0]}   & {ips[j[0]]}  & {j[1]}  & {j[2]}\\\ ")
                else:
                    # print(j)
                    print(f"hop = {count}  IP = {j[0]}   No rDNS  TTL = {j[1]} rtt = {j[2]}",file=df)
                    print(f"{count} & {j[0]} & * &{j[1]} & {j[2]}\\\ ") 
                count+=1  

In [ ]:
import requests
import json
from Mylib import CreateMeasurement_traceroute

URL_SCHEDUL = "https://atlas.ripe.net/api/v2/measurements//?key="
KEY_ACESS = "58e3081e-9e4c-42c1-8c15-9c8b103cabf9"
ips = list()
with open("IP_2_traceroute",'r') as fileips:
    with open("measurement_traceroute",'a') as fl:
        for i in fileips:
            a = i.split(' ')
            result = CreateMeasurement_traceroute(a[0])
            print(f"{result} {a[2][:-1]}",file=fl)



In [ ]:
####################### Busca resultados de measurements já criado no RIPE  #####################

import requests
import json
import sqlite3
import multiprocessing as mp
import os
from datetime import datetime,timedelta
from datetime import date
import time
import bisect
import tqdm

def traceroute_result(measurement:int):
    # target = dict()    
    url = 'https://atlas.ripe.net:443/api/v2/measurements/'
    resposta = requests.get(url+str(measurement)+"/results")
    resposta = resposta.json()
    target = dict()
    for probe in resposta:
        target[probe["from"]] = list()
        for result1 in probe["result"]:
            for result in result1["result"]:
                if "from" in result:
                    target[probe["from"]].append((result["from"],result["ttl"]))
                    ips0.add(result["from"])
                else:
                    target[probe["from"]].append((result["x"],result["x"]))
                break
    return(probe["dst_addr"],target)

ips0 = set()

a = list()
with open("measurement_traceroute",'r') as fp:
    for i in fp:
        r = i.split(" ")
        a.append((r[0],r[1]))

target = dict() 
for measurement,dns in tqdm.tqdm(a):
    saida = traceroute_result(measurement)
    target[saida[0]] = saida[1]

In [ ]:
import requests
import tqdm

a = list()
url = 'https://atlas.ripe.net:443/api/v2/measurements/'
with open("measurement_traceroute",'r') as fl1:
    for each in fl1:
        measurements = each.split(' ')
        a.append((measurements[0],measurements[1]))
with open("all_measurements_traceroutes",'a') as fl2:
    for measurement,dns in tqdm.tqdm(a):
        target = dict()    
        resposta = requests.get(url+str(measurement)+"/results")
        resposta = resposta.json()
        for probe in resposta:
            fl2.write(probe["dst_name"] +" "+str(probe["prb_id"])+" ")
            for result1 in probe["result"]:
                for result in result1["result"]:
                    if "from" in result:
                        fl2.write(result["from"]+" "+str(result["ttl"])+" "+str(result["rtt"])+" ")

                    else:
                        fl2.write(result["x"]+" "+result["x"]+" "+result["x"] +" ")
                    break
            fl2.write('\n')

In [ ]:
class cluster:
    tam_sufix:int
    sufix:str
    ips:list
    quant:int
    def __init__(self,sufi,tam,ip):
        self.sufix = sufi
        self.tam_sufix = tam
        self.quant = 1
        self.ips = list()
        self.ips.append(ip)
    def add(self,ip):
        self.ips.append(ip)
        self.quant+=1
    def sufixo(self):
        return self.sufix

def is_same_PoP(ips,one,two):
    ips_list = set()
    ips_list.add(two)
    ips_list.add(one)
    print(ips_list)
    clusters = list()
    for j in range(10,1,-1):
        rm_set = set()
        for i in ips_list:
            a = ips[i].split(".")
            tam = len(a)
            if tam >= j:
                sufix = str()
                for count in range(tam+1-j,tam):
                    sufix = sufix + '.' + a[count]
                if len(clusters) == 0:
                    clusters.append(cluster(sufix,tam,i))
                    rm_set.add(i)
                else:
                    quan = len(clusters)
                    index = None
                    for cluster_count in range(quan):
                        if sufix == clusters[cluster_count].sufixo():
                            index = cluster_count
                            break
                    if index is not None:
                        clusters[index].add(i)
                    else:
                        clusters.append(cluster(sufix,tam,i))
                    rm_set.add(i)
        ips_list = ips_list - rm_set
        quan = len(clusters)
        list_rm = list()
        rm = 0
        if(j > 4):
            for cluster_count in range(quan):
                if clusters[cluster_count].quant == 1:
                    ips_list.add(clusters[cluster_count].ips[0])
                    a = clusters[cluster_count]
                    del clusters[cluster_count]
                    clusters.insert(0,a)
                    rm += 1
            for asd in range(rm):
                del clusters[0]
        else:
            break
    for i in clusters:
        print(i.sufix)
    if len(clusters) > 1:
        return False
    else:
        return True

In [ ]:
from Mylib import is_same_PoP
with open("rDNSmaster/outro",'r') as fl:
    ips = dict()
    ips_list = set()
    for i in fl:
        aux  = i.split(" ,")
        ips[aux[0]] = aux[1][:-1].lower()
        ips_list.add(aux[0])
print(is_same_PoP(ips,"213.19.196.241","212.72.33.61"))

Na célula abaixo é usada a funcão analysis_traceroute para comparar os traceroute de uma lista de ips.
(OBS: os traceroutes devem existir e estarem gravados no arquivo all_measurements_traceroutes)

In [ ]:

            
import json
from Mylib import analysis_traceroute
analysis_traceroute(("213.19.196.241",),("4.69.153.194",))
# analysis(("213.19.196.241","4.69.153.194","4.68.70.161","212.73.253.101","62.67.38.49","4.69.143.178","195.50.125.201"),("213.19.196.241",))